## Vessl Service

vessl serverless에서 제공하는 resource를 이용한 모델 서비스 이용 및 구축

In [17]:
import sys, subprocess
print("Python:", sys.executable)

subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "pip", "setuptools", "wheel"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U",
                       "requests", "urllib3", "idna", "certifi", "charset-normalizer"])

import requests, urllib3, idna
print("versions:", requests.__version__, urllib3.__version__, idna.__version__)


Python: c:\Python314\python.exe
versions: 2.32.5 2.5.0 3.11


In [ ]:
# requests 이용

import requests
import urllib3
import base64
import time
import json
import os

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) ## SSL 인증서 경고 무시

# === 설정 ===

RUNPOD_API_KEY = ""  # 외부로 유출 금지
ENDPOINT_ID = "n4q9uzty70zzsu"                  # ← RunPod Serverless Endpoint ID
API_URL = f"https://api.runpod.ai/v2/{ENDPOINT_ID}/runsync"

PDB_FILE_PATH = "./inputs/insulin_target.pdb"

# RFdiffusion 파라미터 (리스트로)
commands = [
    "contigmap.contigs=[A1-150/0 70-100]",
    "ppi.hotspot_res=[A59,A83,A91]",
    "inference.num_designs=3",
    "denoiser.noise_scale_ca=0",
    "denoiser.noise_scale_frame=0"
]

# RFdiffusion 파라미터 (리스트로)
commands = [
    "contigmap.contigs=[A1-150/0 70-100]",
    "ppi.hotspot_res=[A59,A83,A91]",
    "inference.num_designs=3",
    "denoiser.noise_scale_ca=0",
    "denoiser.noise_scale_frame=0"
]



In [ ]:
# === 1. PDB 파일 읽기 및 base64 인코딩 ===
with open(PDB_FILE_PATH, "rb") as f:
    pdb_b64 = base64.b64encode(f.read()).decode("utf-8")

# === 2. RunPod에 요청 전송 ===
payload = {
    "input": {
        "pdb_file": pdb_b64,
        "commands": commands,
        "model_directory_path": "/runpod-volume/rfdiffusion_models"  # 네트워크 볼륨 캐시 권장
    }
}

headers = {
    "Authorization": f"Bearer {RUNPOD_API_KEY}",
    "Content-Type": "application/json"
}

# 간단한 유효성 검사
if not RUNPOD_API_KEY:
    raise RuntimeError("RUNPOD_API_KEY 환경변수가 비어 있습니다. os.environ['RUNPOD_API_KEY']='rpa_...' 로 설정하세요.")
auth_preview = (RUNPOD_API_KEY[:6] + '...' + RUNPOD_API_KEY[-4:]) if len(RUNPOD_API_KEY) > 10 else 'set'
print(f"Endpoint: {ENDPOINT_ID}, Auth: {auth_preview}")

print("🚀 Sending job to RunPod /runsync...")
try:
    response = requests.post(API_URL, json=payload, headers=headers, timeout=1800, verify=False)  # 최대 30분
    response.raise_for_status()
except requests.Timeout:
    print("❌ Request timed out. Consider using /run + polling for long jobs.")
    raise
except requests.HTTPError as e:
    print("❌ HTTPError:", e)
    if e.response is not None:
        print("Status:", e.response.status_code)
        print("Body:", e.response.text[:1000])
    raise

# === 3. 응답 처리 ===
if response.status_code == 200:
    data = response.json()
    out = data.get("output", data)
    if isinstance(out, dict) and "error" in out:
        print("❌ Job failed with error:")
        print(json.dumps(out, indent=2))
    else:
        # === 4. 결과 다운로드 (base64 → ZIP) ===
        zip_b64 = out.get("result_zip_b64") if isinstance(out, dict) else None
        if zip_b64:
            output_path = "rfdiffusion_result.zip"
            with open(output_path, "wb") as f:
                f.write(base64.b64decode(zip_b64))
            print(f"✅ Success! Output saved to: {output_path}")
        else:
            print("⚠️ No result_zip_b64 in response. Full output follows.")
            print(json.dumps(data, indent=2)[:2000])
        # 로그 일부 출력
        if isinstance(out, dict):
            print("STDOUT:", (out.get("stdout", "") or "")[:1000])
            print("STDERR:", (out.get("stderr", "") or "")[:1000])
else:
    print(f"❌ API request failed: {response.status_code}")
    print(response.text)